In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),  
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=0)
val_loader = DataLoader(test_set, batch_size=64, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.25)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.batchnorm3 = nn.BatchNorm2d(128)

    def forward(self, x):
        x = self.pool(F.relu(self.batchnorm1(self.conv1(x))))
        x = self.pool(F.relu(self.batchnorm2(self.conv2(x))))
        x = self.pool(F.relu(self.batchnorm3(self.conv3(x))))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)


model = CNN().to(device)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [6]:
num_epochs = 100

In [7]:
# Early stopping parameters
patience = 5
best_loss = float('inf')
epochs_no_improve = 0
epsilon = 1e-3
# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation step
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}")
    
    # Early Stopping
    if val_loss < best_loss-epsilon:
        best_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f'Early stopping at epoch {epoch + 1}')
            break


Epoch [1/100], Validation Loss: 1.6285
Epoch [2/100], Validation Loss: 1.4025
Epoch [3/100], Validation Loss: 1.3024
Epoch [4/100], Validation Loss: 1.2039
Epoch [5/100], Validation Loss: 1.1163
Epoch [6/100], Validation Loss: 1.0680
Epoch [7/100], Validation Loss: 1.0556
Epoch [8/100], Validation Loss: 1.0111
Epoch [9/100], Validation Loss: 0.9604
Epoch [10/100], Validation Loss: 0.9391
Epoch [11/100], Validation Loss: 0.9053
Epoch [12/100], Validation Loss: 0.8965
Epoch [13/100], Validation Loss: 0.8620
Epoch [14/100], Validation Loss: 0.8393
Epoch [15/100], Validation Loss: 0.8169
Epoch [16/100], Validation Loss: 0.8313
Epoch [17/100], Validation Loss: 0.8151
Epoch [18/100], Validation Loss: 0.8218
Epoch [19/100], Validation Loss: 0.7941
Epoch [20/100], Validation Loss: 0.7933
Epoch [21/100], Validation Loss: 0.7786
Epoch [22/100], Validation Loss: 0.7551
Epoch [23/100], Validation Loss: 0.7522
Epoch [24/100], Validation Loss: 0.7367
Epoch [25/100], Validation Loss: 0.7259
Epoch [26

### Evaluating the supervised learning model

In [8]:
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


Accuracy of the network on the 10000 test images: 81 %


In [ ]:
torch.save(model.state_dict(), 'cnn.pth')